<a href="https://colab.research.google.com/github/LD-Shell/Chemical_Engineering/blob/main/CHEE_6331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re

### Row echelon form implementation

In [ ]:
matrix_A = np.array([[-6,2, -2],
            [2, -6, -2],
            [-2,-2, -2],
            ])

In [ ]:
print("The Rank of a Matrix: ", np.linalg.matrix_rank(matrix_A))

The Rank of a Matrix:  2


In [ ]:
def is_row_echelon_form(matrix: np.ndarray) -> bool:
    """Checks if a matrix is in row echelon form.

    Args:
        matrix: The input NumPy matrix.

    Returns:
        True if the matrix is in row echelon form, False otherwise.
    """

    if not matrix.any():
        return False

    num_rows, num_cols = matrix.shape
    prev_leading_col = -1

    for row in range(num_rows):
        leading_col_found = False
        for col in range(num_cols):
            if matrix[row, col] != 0:
                if col <= prev_leading_col:
                    return False
                prev_leading_col = col
                leading_col_found = True
                break
        if not leading_col_found and any(matrix[row, col] != 0 for col in range(num_cols)):
            return False
    return True

def find_nonzero_row(matrix: np.ndarray, pivot_row: int, col: int) -> int | None:
    """Finds the first non-zero row below the pivot row in a column.

    Args:
        matrix: The input NumPy matrix.
        pivot_row: The current pivot row.
        col: The column to search.

    Returns:
        The index of the first non-zero row, or None if no such row exists.
    """

    num_rows = matrix.shape[0]
    for row in range(pivot_row, num_rows):
        if matrix[row, col] != 0:
            return row
    return None

def swap_rows(matrix: np.ndarray, row1: int, row2: int):
    """Swaps two rows of a NumPy matrix.

    Args:
        matrix: The input NumPy matrix.
        row1: The index of the first row.
        row2: The index of the second row.
    """

    matrix[[row1, row2]] = matrix[[row2, row1]]

def make_pivot_one(matrix: np.ndarray, pivot_row: int, col: int) -> bool:
    """Makes the pivot element in a row equal to 1.

    Args:
        matrix: The input NumPy matrix.
        pivot_row: The row containing the pivot element.
        col: The column containing the pivot element.

    Returns:
        True if the pivot element was successfully made 1, False if the pivot element is effectively zero.
    """

    pivot_element = matrix[pivot_row, col]
    if abs(pivot_element) < 1e-12:  # Adjust tolerance as needed
        return False  # Pivot element is effectively zero
    for j in range(col, matrix.shape[1]):
        matrix[pivot_row, j] /= pivot_element
    return True

def eliminate_below(matrix: np.ndarray, pivot_row: int, col: int):
    """Eliminates elements below the pivot element in a column.

    Args:
        matrix: The input NumPy matrix.
        pivot_row: The row containing the pivot element.
        col: The column containing the pivot element.
    """

    num_rows = matrix.shape[0]
    pivot_element = matrix[pivot_row, col]
    for row in range(pivot_row + 1, num_rows):
        factor = matrix[row, col]
        matrix[row] -= factor * matrix[pivot_row]

def row_echelon_form(matrix: np.ndarray) -> np.ndarray:
    """Reduces a NumPy matrix to row echelon form.

    Args:
        matrix: The input NumPy matrix.

    Returns:
        The reduced row echelon form of the matrix.
    """

    matrix = matrix.astype(np.float64)
    num_rows, num_cols = matrix.shape
    pivot_row = 0

    for col in range(num_cols):
        # Find the row with the largest absolute value in the current column
        max_row = pivot_row
        max_val = abs(matrix[pivot_row, col])
        for row in range(pivot_row + 1, num_rows):
            if abs(matrix[row, col]) > max_val:
                max_row = row
                max_val = abs(matrix[row, col])

        # Swap rows if necessary
        if max_row != pivot_row:
            swap_rows(matrix, pivot_row, max_row)

        # Make pivot element one
        if not make_pivot_one(matrix, pivot_row, col):
            continue  # Pivot element is effectively zero, move to next column

        # Eliminate elements below the pivot
        eliminate_below(matrix, pivot_row, col)
        pivot_row += 1

    return matrix

In [ ]:
is_row_echelon_form(matrix_A)

False

In [ ]:
print("Matrix Before Converting:")
print(matrix_A)
print()
result = row_echelon_form(matrix_A)
print("After Converting to Row Echelon Form:")
print(result)
if is_row_echelon_form(result):
    print("In REF")
else:
    print("Not in REF--------------->")

Matrix Before Converting:
[[-6  2 -2]
 [ 2 -6 -2]
 [-2 -2 -2]]

After Converting to Row Echelon Form:
[[ 1.00000000e+00 -3.33333333e-01  3.33333333e-01]
 [ 0.00000000e+00  1.00000000e+00  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
In REF


### Application of vector expansion to stiochiometry

#### _main function_

In [ ]:
# Function to parse a chemical formula and count the atoms
def count_atoms(formula):
    # Regular expression to match elements and their quantities
    matches = re.findall(r'([A-Z][a-z]?)(\d*)', formula)

    # Dictionary to store atom counts
    atom_counts = {}

    for element, count in matches:
        if count == '':  # If no number is specified, it means 1 atom
            count = 1
        else:
            count = int(count)

        if element in atom_counts:
            atom_counts[element] += count
        else:
            atom_counts[element] = count

    return atom_counts

# Function to generate a list of unique elements
def get_unique_elements(formulas):
    unique_elements = set()

    for formula in formulas:
        matches = re.findall(r'([A-Z][a-z]?)', formula)
        unique_elements.update(matches)

    return sorted(unique_elements)

# Function to generate a dataframe from a list of formulas
def generate_dataframe(formulas):
    all_atom_counts = []

    # List of all elements that may appear (extend this list as needed)
    elements = get_unique_elements(formulas)

    # Process each formula
    for formula in formulas:
        atom_counts = count_atoms(formula)
        row = {element: atom_counts.get(element, 0) for element in elements}
        all_atom_counts.append(row)

    # Create a pandas dataframe
    df = pd.DataFrame(all_atom_counts, index=formulas)

    return df

#### _Example usage:_

In [ ]:
formulas = ('CH4', 'C2H6', 'H2O', 'H2', 'O2', 'CO', 'CO2', 'C2H4', 'C2H2')
df = generate_dataframe(formulas)

# Print the resulting dataframe
print(df)

In [ ]:
matrix = df.T.to_numpy()

# Print the resulting numpy matrix
print(matrix)
print("The Rank of a Matrix: ", np.linalg.matrix_rank(matrix))

#### _assignment implementatiion_

In [ ]:
formulas_hw = ('NO', 'N2O', 'NO2', 'N2', 'NH3', 'H2O', 'H2', 'O2')
df_hw = generate_dataframe(formulas_hw)

# Print the resulting dataframe
print(df_hw)

matrix_hw = df_hw.T.to_numpy()

# Print the resulting numpy matrix
print(matrix_hw)
print("The Rank of a Matrix: ", np.linalg.matrix_rank(matrix_hw))